In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import osmnx as ox

In [2]:
city = 'chicago'
city_crs = {'seattle': 2285, '' + city + '': 3424, 'chicago': 3435}

In [13]:
#read in road network
road = gpd.read_file('../../../data/' + city + '/osm/osm-roads/streets-with-osm-way-type.shp')
road

,street_edg,osm_id,way_type,geometry
0,320,5015170,secondary,"LINESTRING (-87.62392 41.85789, -87.62441 41.8..."
1,321,5015170,secondary,"LINESTRING (-87.62562 41.85787, -87.62625 41.8..."
2,322,5015170,secondary,"LINESTRING (-87.62715 41.85783, -87.62730 41.8..."
3,326,5043580,primary,"LINESTRING (-87.62367 41.84748, -87.62373 41.8..."
4,327,5043581,residential,"LINESTRING (-87.62232 41.86093, -87.62232 41.8..."
...,...,...,...,...
4785,331271,965104703,residential,"LINESTRING (-87.62490 41.82739, -87.62487 41.8..."
4786,331272,965104703,residential,"LINESTRING (-87.62487 41.82648, -87.62486 41.8..."
4787,331273,965104704,residential,"LINESTRING (-87.62480 41.82386, -87.62482 41.8..."
4788,331274,965104704,residential,"LINESTRING (-87.62482 41.82373, -87.62483 41.8..."


In [14]:
#read in labels
labels = gpd.read_file('../../../data/' + city + '/processed-labels/labels_raw_' + city + '/labels_raw_' + city + '.shp')
labels

,audit_task,label_id,gsv_panora,label_type,severity,correct,high_quali,gsv_pano_1,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry
0,3,8,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,None,1,pMph48Z6Xz2hexFJ7q3d_Q,2,136.687500,-18.625000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083668,-87.980347,POINT (-87.98035 42.08367)
1,3,9,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,1.0,1,1,pMph48Z6Xz2hexFJ7q3d_Q,2,249.937500,-14.125000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083694,-87.980606,POINT (-87.98061 42.08369)
2,3,10,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,0,1,pMph48Z6Xz2hexFJ7q3d_Q,3,288.656250,-13.187500,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083786,-87.980606,POINT (-87.98061 42.08379)
3,3,11,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,1,1,pMph48Z6Xz2hexFJ7q3d_Q,3,49.245537,-16.220982,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083794,-87.980354,POINT (-87.98035 42.08379)
4,3,12,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,1,1,pMph48Z6Xz2hexFJ7q3d_Q,3,42.636162,-16.033482,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083817,-87.980362,POINT (-87.98036 42.08382)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,4588,21507,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,3.0,1,1,hhAF_qDTHgN0dL8SNsr-oA,1,126.341515,-32.937500,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611179,-87.664291,POINT (-87.66429 41.61118)
18281,4588,21508,hhAF_qDTHgN0dL8SNsr-oA,SurfaceProblem,2.0,None,1,hhAF_qDTHgN0dL8SNsr-oA,1,213.154022,-35.000000,179.073898,0.395355,71959a82-3f47-4dec-a02b-86669bf8095b,41.611137,-87.664551,POINT (-87.66455 41.61114)
18282,4589,21509,tZWhmOw4CIP9bQLJycSyqQ,CurbRamp,1.0,None,1,tZWhmOw4CIP9bQLJycSyqQ,3,217.380630,-16.302500,179.386520,-0.660645,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,41.609871,-87.646339,POINT (-87.64634 41.60987)
18283,4601,21517,nWTcoxo4S0MImzzvlQGwpQ,CurbRamp,1.0,1,1,nWTcoxo4S0MImzzvlQGwpQ,3,248.244995,-13.510625,359.491150,-0.079690,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,41.625626,-87.651291,POINT (-87.65129 41.62563)


In [15]:
#change coordinates
labels = labels.to_crs(epsg=city_crs[city])
road = road.to_crs(epsg=city_crs[city])

In [16]:
#spatial join project sidewalk labels to road geometry
labels_mapped_on_road = labels.sjoin_nearest(road, distance_col='distance_to_road')
labels_mapped_on_road

,audit_task,label_id,gsv_panora,label_type,severity,correct,high_quali,gsv_pano_1,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,index_right,street_edg,osm_id,way_type,distance_to_road
0,3,8,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,2.0,None,1,pMph48Z6Xz2hexFJ7q3d_Q,2,136.687500,-18.625000,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083668,-87.980347,POINT (1080069.519 1973277.133),765,117289,24083797,residential,26.799369
3,3,11,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,1,1,pMph48Z6Xz2hexFJ7q3d_Q,3,49.245537,-16.220982,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083794,-87.980354,POINT (1080067.259 1973322.999),765,117289,24083797,residential,19.033046
4,3,12,pMph48Z6Xz2hexFJ7q3d_Q,CurbRamp,3.0,1,1,pMph48Z6Xz2hexFJ7q3d_Q,3,42.636162,-16.033482,268.225098,-0.562027,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083817,-87.980362,POINT (1080065.153 1973331.331),765,117289,24083797,residential,27.337529
7,3,15,JNAmJAZYxrX7YSC4C3EwtA,SurfaceProblem,1.0,1,1,JNAmJAZYxrX7YSC4C3EwtA,3,33.604912,-15.194197,269.981445,0.073532,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083820,-87.979927,POINT (1080183.197 1973333.209),765,117289,24083797,residential,30.729777
8,5,16,VvFlgEkOuGmMtvvgO6lZog,CurbRamp,1.0,1,1,VvFlgEkOuGmMtvvgO6lZog,2,315.229919,-14.551339,270.559692,1.520401,3e25ea1d-f762-421a-b98b-3cba36e1bbfb,42.083771,-87.979370,POINT (1080334.457 1973315.763),765,117289,24083797,residential,15.248823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18267,4588,21492,g2bClkWMQjCLoNVtPMkLUA,CurbRamp,3.0,1,1,g2bClkWMQjCLoNVtPMkLUA,1,286.091522,-35.000000,179.339081,0.205020,71959a82-3f47-4dec-a02b-86669bf8095b,41.611813,-87.664490,POINT (1167150.996 1801847.047),610,112608,24081394,residential,20.654522
18269,4588,21495,g2bClkWMQjCLoNVtPMkLUA,CurbRamp,2.0,None,1,g2bClkWMQjCLoNVtPMkLUA,1,39.341518,-30.312500,179.339081,0.205020,71959a82-3f47-4dec-a02b-86669bf8095b,41.611816,-87.664345,POINT (1167190.626 1801848.744),610,112608,24081394,residential,18.993256
18258,4572,21483,pHSPCrFgfI1xNH2xUJZEtw,CurbRamp,2.0,None,1,pHSPCrFgfI1xNH2xUJZEtw,1,208.841522,-34.625000,176.231155,0.248665,71959a82-3f47-4dec-a02b-86669bf8095b,41.613483,-87.664543,POINT (1167131.670 1802455.760),4065,286599,577599281,residential,23.923173
18268,4588,21494,g2bClkWMQjCLoNVtPMkLUA,CurbRamp,3.0,None,1,g2bClkWMQjCLoNVtPMkLUA,1,286.091522,-35.000000,179.339081,0.205020,71959a82-3f47-4dec-a02b-86669bf8095b,41.611771,-87.664543,POINT (1167136.510 1801831.644),1520,145319,24100605,residential,17.028374


In [17]:
#select only label_id, distance
distance = labels_mapped_on_road[['label_id','way_type', 'distance_to_road']]

In [18]:
#drop duplicates
distance = distance.drop_duplicates(subset=['label_id'], keep='first')

In [19]:
# reset index
distance = distance.reset_index(drop=True)

In [20]:
distance

,label_id,way_type,distance_to_road
0,8,residential,26.799369
1,11,residential,19.033046
2,12,residential,27.337529
3,15,residential,30.729777
4,16,residential,15.248823
...,...,...,...
18280,21492,residential,20.654522
18281,21495,residential,18.993256
18282,21483,residential,23.923173
18283,21494,residential,17.028374


In [21]:
# save distance into a csv file
distance.to_csv('../../../data/' + city + '/features/distance_to_road.csv', index=False)